In [84]:
import pandas as pd
import numpy as np

In [85]:
import bokeh.plotting as bpl
import bokeh.models as bmo
from bokeh.palettes import d3
bpl.output_notebook()

Loading BokehJS ...

In [86]:
df = pd.read_csv('device_presence.log', sep=' ', names=['Day', 'Time', 'd', 'is', 'Value'])

In [87]:
df = df[['Day', 'Time', 'Value']]

In [88]:
df['Time'] = df['Time'].str[:8]

In [89]:
def nonlin(x,deriv=False):
# sigmoid function
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

# Sample Data

In [121]:
temp = df

In [122]:
temp['time'] = temp.Time.apply(lambda x: x[:2].replace(':', ''))
temp['value'] = temp.Value.apply(lambda x: 1 if x == 'online' else 0)

temp['weekday'] = pd.to_datetime(temp['Day']).dt.dayofweek

In [119]:
np.random.seed(1)

w0 = 2*np.random.random((2,1)) - 1
#w1 = 2*np.random.random(())

for i in range(100):
    X  = temp[['weekday', 'time']]
    L0 = np.array(X, dtype=int)
    Y  = np.array(temp[['value']], dtype=int)
    
    L1 = nonlin(np.dot(L0, w0))
    
    L0_error = Y - L1
    
    if (i % 10 == 0):
        print('Error after iteration {i}: {error}'.format(i=i, error=np.mean(np.abs(L0_error))))
        print(L1)
    
    L0_delta = L0_error * nonlin(L1, True)
    
    w0 += np.dot(L0.T, L0_delta)
    

Error after iteration 0: 0.6747884394041918
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
Error after iteration 10: 0.3253802504222913
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Error after iteration 20: 0.32538025035252727
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Error after iteration 30: 0.32538025028114054
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Error after iteration 40: 0.3253802502080737
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Error after iteration 50: 0.325380250133267
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Error after iteration 60: 0.32538025005665744
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Error after iteration 70: 0.325380249978179
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Error after iteration 80: 0.32538024989776265
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Error after iteration 90: 0.32538024981533553
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [118]:
nonlin(np.dot([4, 2316], w0))

array([0.])

In [123]:
temp[['weekday', 'time', 'Value']]

,weekday,time,Value
0,5,14,offline
1,5,14,offline
2,5,14,offline
3,5,14,offline
4,5,14,offline
5,5,14,offline
6,5,14,offline
7,5,14,offline
8,5,14,offline
9,5,14,offline


# Visualization

In [ ]:
temp['Datetime']= temp['Day'].str.cat(temp['Time'], sep=' ')
temp['Datetime'] = pandas.to_datetime(temp['Datetime'])

In [ ]:
temp = temp[['Datetime', 'Value']]

In [ ]:
temp = temp.set_index('Datetime')

In [ ]:
temp['Value'] = temp.Value.apply(lambda x: 1 if x == 'online' else -1)

In [ ]:
temp = temp.sort_index()

In [ ]:
#temp_week = temp[:'2018-04-27']
temp_week = temp['2018-04-28':]

In [ ]:
temp_week = temp_week.resample('1H').apply(lambda x: 1 if sum(x) > 0 else -1)

In [ ]:
is_online = temp_week['Value']==1

In [ ]:
temp_week['Date'] = temp_week.index.date
temp_week['Time'] = temp_week.index.time

In [ ]:
p = bpl.figure(width=800, height=600, tools="pan,box_zoom,reset,save", y_axis_label='Day', x_axis_label='Time', y_axis_type='datetime', x_axis_type='datetime')

In [ ]:
p.circle(temp_week['Time'][is_online], temp_week['Date'][is_online], fill_color='green', size=8)
p.circle(temp_week['Time'][~is_online], temp_week['Date'][~is_online], fill_color='red', size=8)

In [ ]:
p.yaxis.formatter = bmo.formatters.DatetimeTickFormatter(days='%a')

In [ ]:
bpl.show(p)

In [ ]:
len(temp_week)

temp

In [ ]:
temp_week